<a href="https://colab.research.google.com/github/By0ungJoo/dacon/blob/main/image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import shutil # high-level file operations
from tqdm import tqdm
from glob import glob
import re
import cv2
import gc

from sklearn.model_selection import train_test_split

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform

In [4]:


## 주어진 train_df에 각 레이블마다 원-핫인코딩값을 생성할 인덱스 부여
train_df = pd.read_csv('/content/drive/MyDrive/DACON_이상치 탐지 알고리즘 경진대회/open/train_df.csv')
final_train88_df = train_df.copy()
label_lst = final_train88_df.label.unique().tolist()
label_lst.sort()
one_hot_label = pd.DataFrame(label_lst, columns = {'label'})
one_hot_label['one_hot_label'] = one_hot_label.index.tolist()
one_hot_label

final_train88_df = pd.merge(final_train88_df, one_hot_label, how = 'left', on = 'label')
final_train88_df

# 학습 파일 기본 경로 지정 및 넘파이 배열로 변환
# 넘파일 배열을 생성할 레이블의 one_hot_label값 [0 ~ 87] / df > one_hot_label 참조
image_dir = '/content/drive/MyDrive/dacon/dacon/open/train'
num_total_label = 88 # 레이블 총 개수
one_hot_label = [x for x in range(88)]

img_rows = 224
img_cols = 224

X = [] # 입력 데이터
Y = [] # 정답값

## 이미지 배열 파일 생성
for index, row in tqdm(final_train88_df.iterrows(), total = len(final_train88_df)) : 
  filename = row['file_name']
  label_idx = row['one_hot_label']
  # img_label : Y 배열에 들어갈 원-핫 인코딩된 정답값
  img_label = [0 for i in range(num_total_label)]
  img_label[label_idx] = 1

  img = cv2.imread(f'{image_dir}/{filename}').astype('float32')
  img = cv2.resize(img, (img_rows,img_cols))#, fx=img_w/img.shape[1], fy=img_h/img.shape[0])
  X.append(img/256)
  Y.append(img_label)

X = np.array(X)
Y = np.array(Y)

x_train, x_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.3, random_state=1234)

## 안쓰는 변수 비우기 : 램관리
del X, Y
gc.collect()

100%|██████████| 4277/4277 [02:16<00:00, 31.34it/s]


11

In [5]:
def bottleneck_residual_block(X, f, filters, stage, block, reduce=False, s=2):
    """    
    Arguments:
    X -- input tensor of shape (m, height, width, channels)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    reduce -- boolean, True = identifies the reduction layer at the beginning of each learning stage
    s -- integer, strides
    
    Returns:
    X -- output of the identity block, tensor of shape (H, W, C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    if reduce:
        # if we are to reduce the spatial size, apply a 1x1 CONV layer to the shortcut path
        # to do that, we need both CONV layers to have similar strides 
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)
        
        X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    else: 
        # First component of main path
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [6]:
def ResNet50(input_shape, classes):
    """
    Arguments:
    input_shape -- tuple shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='a', reduce=True, s=1)
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='e')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='f')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='g')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='h')
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='i')

    # Stage 4 
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='g')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='h')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='i')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='j')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='k')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='l')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='m')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='n')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='o')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='p')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='q')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='r')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='s')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='t')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='u')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='v')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='w')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='x')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='y')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='z')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='aa')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ab')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ac')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ad')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ae')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='af')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ag')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ah')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ai')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='aj')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='ak')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='al')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='am')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='an')

    # Stage 5 
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL 
    X = AveragePooling2D((1,1), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create the model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [11]:
model = ResNet50(input_shape = (224, 224, 3), classes = 88)

In [12]:
model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [13]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/dacon/dacon/model1.h5', monitor='loss', save_best_only=True)
# patience 5
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

In [ ]:
history = model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks =[checkpoint_cb, early_stopping_cb])

Epoch 1/100
19/47 [===========>..................] - ETA: 6:44 - loss: 26.1260 - accuracy: 0.1242

In [ ]:
# 5. 학습 과정 살펴보기
%matplotlib inline
import matplotlib.pyplot as plt

## 그래프 : loss , val_loss
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

## loss 그래프
loss_ax.plot(history.history['loss'],'y',label='train_loss') # 노란색
loss_ax.plot(history.history['val_loss'],color='red' ,label='val_loss') # 빨간색

## accuracy 그래프
acc_ax.plot(history.history['accuracy'],'g',label='train_accuracy') # 초록색
acc_ax.plot(history.history['val_accuracy'],'b',label='val_accuracy') # 파란색

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()

In [ ]:
# epoch 100 으로 코드 돌리기 